In [ ]:
import os
import random
import shutil

# Define paths
base_folders = ["dataset/actor_faces", "dataset/actress_faces"]
train_folder = "dataset/stagging/sampleset"
test_folder = "dataset/stagging/testingset"

# Function to clear and recreate a folder
def reset_folder(folder_path):
    if os.path.exists(folder_path):
        shutil.rmtree(folder_path)  # Delete everything inside
    os.makedirs(folder_path)  # Recreate the empty folder

# Reset train and test folders
reset_folder(train_folder)
reset_folder(test_folder)
# Collect all person folders from both `actor_faces` and `actress_faces`
all_persons = []
for base_folder in base_folders:
    if os.path.exists(base_folder):  # Ensure the folder exists before listing
        persons = [os.path.join(base_folder, p) for p in os.listdir(base_folder) if os.path.isdir(os.path.join(base_folder, p))]
        all_persons.extend(persons)  # Add full paths

#print(all_persons)
# Randomly select up to 200 persons
selected_persons = random.sample(all_persons, min(200, len(all_persons)))
print(f"Selected {len(selected_persons)} persons.")

# Process each selected person
for person_path in selected_persons:
    person_name = os.path.basename(person_path)  # Extract only folder name

    images = [img for img in os.listdir(person_path) if img.endswith((".jpeg", ".jpg", ".png"))]

    if len(images) < 20:  # Skip if not enough images
        print(f"Skipping {person_name}, not enough images.")
        continue

    # Shuffle and select max 25 images
    random.shuffle(images)
    selected_images = images[:25]

    # Split into train (20) and test (5)
    train_images = selected_images[:20]
    test_images = selected_images[20:]

    # Create person folders in train and test
    train_person_path = os.path.join(train_folder, person_name)
    test_person_path = os.path.join(test_folder, person_name)
    os.makedirs(train_person_path, exist_ok=True)
    os.makedirs(test_person_path, exist_ok=True)

    # Move images
    for img in train_images:
        shutil.copy(os.path.join(person_path, img), os.path.join(train_person_path, img))

    for img in test_images:
        shutil.copy(os.path.join(person_path, img), os.path.join(test_person_path, img))

    print(f"Processed {person_name}: Train={len(train_images)}, Test={len(test_images)}")

print("✅ Dataset organized successfully!")


Selected 200 persons.
Processed Kassie_DePaiva: Train=20, Test=5
Processed Bradley_Cooper: Train=20, Test=5
Processed Cam_Gigandet: Train=20, Test=5
Processed Audra_McDonald: Train=20, Test=5
Processed Shannon_Kane: Train=20, Test=5
Processed Crystal_Chappell: Train=20, Test=5
Processed Jennette_McCurdy: Train=20, Test=5
Processed John_Malkovich: Train=20, Test=5
Processed Seth_Rogen: Train=20, Test=5
Processed Kris_Kristofferson: Train=20, Test=5
Processed John_Noble: Train=20, Test=5
Processed Catherine_Bell: Train=20, Test=5
Processed Sean_Bean: Train=20, Test=5
Processed Alyssa_Milano: Train=20, Test=5
Processed Jason_Behr: Train=20, Test=5
Processed Martin_Sheen: Train=20, Test=5
Processed Antonio_Banderas: Train=20, Test=5
Processed Tempestt_Bledsoe: Train=20, Test=5
Processed Ed_Harris: Train=20, Test=5
Processed Valerie_Harper: Train=20, Test=5
Processed Amaury_Nolasco: Train=20, Test=5
Processed Oliver_Platt: Train=20, Test=5
Processed Chyler_Leigh: Train=20, Test=5
Processed 

In [3]:
import os
import cv2
import tensorflow as tf
from mtcnn import MTCNN
from imutils import paths
from tqdm import tqdm

In [4]:
from pathlib import Path
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.compat.v1.Session(config=config)

In [29]:
def processed_mtcnn(detector,dirpath, dirdest):
    
    imagePaths = sorted(list(paths.list_images(dirpath)))
    for imagePath in tqdm(imagePaths):
        #path_split = imagePath.split(os.sep)
        path_split=Path(imagePath).parts
        name_person = path_split[-2]
        fn = path_split[-1].split('.')
        filename, fileformat = fn[0], fn[1]

        os.makedirs(dirdest, exist_ok=True)

        image = cv2.cvtColor(cv2.imread(imagePath), cv2.COLOR_BGR2RGB)
        result = detector.detect_faces(image)
        print(f"Processing: {imagePath}, Faces detected: {len(result)}")

        for i in range(len(result)):
            bounding_box = result[i]['box']
            keypoints = result[i]['keypoints']

            bounding_box[0] = max(0, bounding_box[0])
            bounding_box[1] = max(0, bounding_box[1])

            person_dir = os.path.join(dirdest, name_person)
            os.makedirs(person_dir, exist_ok=True)
            path_save = os.path.join(person_dir, f"{filename}_{i}.{fileformat}")
            print(path_save)
            img = image[bounding_box[1]:bounding_box[1] + bounding_box[3],
                    bounding_box[0]:bounding_box[0] + bounding_box[2]]

            cv2.imwrite(path_save, cv2.cvtColor(img, cv2.COLOR_RGB2BGR))


    print("Face detection and cropping completed!")

In [30]:
dirpaths=['dataset/stagging/sampleset','dataset/stagging/testingset']

for dirpath in dirpaths:
    processed_mtcnn(detector=MTCNN(),dirpath=dirpath,dirdest=dirpath.replace("stagging","pcs_mtcnn_faces"))

  0%|          | 1/3980 [00:00<18:10,  3.65it/s]

Processing: dataset/stagging/sampleset\Adrienne_Barbeau\Adrienne_Barbeau_9044_4231.jpeg, Faces detected: 1
dataset/pcs_mtcnn_faces/sampleset\Adrienne_Barbeau\Adrienne_Barbeau_9044_4231_0.jpeg


  0%|          | 2/3980 [00:00<18:02,  3.68it/s]

Processing: dataset/stagging/sampleset\Adrienne_Barbeau\Adrienne_Barbeau_9045_4232.jpeg, Faces detected: 1
dataset/pcs_mtcnn_faces/sampleset\Adrienne_Barbeau\Adrienne_Barbeau_9045_4232_0.jpeg


  0%|          | 3/3980 [00:00<18:59,  3.49it/s]

...

Processing: dataset/stagging/sampleset\Amy_Davidson\Amy_Davidson_39597_18482.jpeg, Faces detected: 1
dataset/pcs_mtcnn_faces/sampleset\Amy_Davidson\Amy_Davidson_39597_18482_0.jpeg


  5%|▌         | 207/3980 [01:31<20:56,  3.00it/s]

Processing: dataset/stagging/sampleset\Amy_Davidson\Amy_Davidson_39598_18483.jpeg, Faces detected: 1
dataset/pcs_mtcnn_faces/sampleset\Amy_Davidson\Amy_Davidson_39598_18483_0.jpeg
Processing: dataset/stagging/sampleset\Amy_Davidson\Amy_Davidson_39602_18486.jpeg, Faces detected: 1
dataset/pcs_mtcnn_faces/sampleset\Amy_Davidson\Amy_Davidson_39602_18486_0.jpeg


  5%|▌         | 207/3980 [01:31<27:54,  2.25it/s]


KeyboardInterrupt: 